### Tabular regression task with FLaVor inference service

* This guide will walk you through tailoring the FLaVor inference service for tabular regression tasks using pytorch.

### Prerequisite

As for the working environment, please ensure you have the following dependencies installed:

```
python >= 3.9
torch >= 2.1.0
```

or simply run:

In [ ]:
!poetry install --with tabular_reg_example

In [ ]:
import os
from typing import Any, Callable, Dict, List, Optional, Sequence

import torch
import torch.nn as nn
import numpy as np
import pandas as pd

from flavor.serve.apps import InferAPP
from flavor.serve.inference.data_models.api import (
    BaseAiCOCOTabularInputDataModel,
    BaseAiCOCOTabularOutputDataModel,
)
from flavor.serve.inference.data_models.functional import AiTable
from flavor.serve.inference.inference_models import BaseAiCOCOTabularInferenceModel
from flavor.serve.inference.strategies import AiCOCOTabularRegressionOutputStrategy

In [ ]:
class SimpleRegressor(nn.Module):
    def __init__(self, input_dim: int = 10, hidden_dim: int = 32, output_dim: int = 1):
        super(SimpleRegressor, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        return self.fc2(x)

In [ ]:
class RegressionInferenceModel(BaseAiCOCOTabularInferenceModel):
    def __init__(self):
        self.formatter = AiCOCOTabularRegressionOutputStrategy()
        super().__init__()

    def define_inference_network(self) -> Callable:
        input_dim = 8  # change this if needed
        model = SimpleRegressor(input_dim=input_dim, hidden_dim=32, output_dim=1)
        model.eval()  # Set the model to evaluation mode.
        return model

    def set_categories(self) -> List[Dict[str, Any]]:
        return None

    def set_regressions(self) -> None:
        regressions = [
            {"name": "reg_value"}
        ]
        return regressions

    def data_reader(self, files: Sequence[str], **kwargs) -> List[pd.DataFrame]:
        file_names = sorted(files, key=lambda s: s[::-1])
        dataframes = [pd.read_csv(file) for file in file_names]
        return dataframes

    def preprocess(self, data: List[pd.DataFrame]) -> pd.DataFrame:
        return pd.concat(data)

    def inference(self, x: pd.DataFrame):
        with torch.no_grad():
            input_tensor = torch.tensor(x.values.astype(np.float32))
            output_tensor = self.network(input_tensor)
            out = output_tensor.numpy().reshape(-1, 1)
        return out

    def postprocess(self, model_out: np.ndarray, **kwargs) -> np.ndarray:
        return model_out

    def output_formatter(
        self,
        model_out: Any,
        tables: Sequence[AiTable],
        dataframes: Sequence[pd.DataFrame],
        meta: Dict[str, Any],
        regressions: Optional[Sequence[Dict[str, Any]]] = None,
        **kwargs,
    ) -> BaseAiCOCOTabularOutputDataModel:

        output = self.formatter(
            model_out=model_out,
            tables=tables,
            dataframes=dataframes,
            regressions=regressions,
            meta=meta,
        )
        return output

In [ ]:
# This block is only for jupyter notebook. You don"t need this in stand-alone script.
import nest_asyncio
nest_asyncio.apply()

In [ ]:
app = InferAPP(
    infer_function=RegressionInferenceModel(),
    input_data_model=BaseAiCOCOTabularInputDataModel,
    output_data_model=BaseAiCOCOTabularOutputDataModel,
)

In [ ]:
app.run(port=int(os.getenv("PORT", 9111)))

### Send request
We can send request to the running server by `send_request.py` which opens the input files and the corresponding JSON file and would be sent via formdata. We expect to have response in AiCOCO tabular format.

```bash
# pwd: examples/inference
python send_request.py -f test_data/tabular/reg/test_reg.csv -d test_data/tabular/reg/input.json
```

## Setup Dockerfile
In order to interact with other services, we have to wrap the inference model into a docker container. 
Here"s an example of the dockerfile. Please put your python dependencies into `requirements.txt` first.

```dockerfile
FROM python:3.9-slim

RUN pip install -r requirements.txt

RUN pip install https://github.com/ailabstw/FLaVor/archive/refs/heads/release/stable.zip

WORKDIR /app

COPY your_script.py  /app/

CMD ["python", "your_script.py"]

```